In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Model test repeatability?

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5
from sklearn.metrics import r2_score

from rolldecay import database
from mdldb.tables import Run
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, OffsetTransformer

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_cubic_b', limit_score=0.9, 
                             only_latest_runs=False)

In [ ]:
df_rolldecay_only_latest = database.load(rolldecay_table_name='rolldecay_cubic_b', limit_score=0.9, 
                             only_latest_runs=True)

In [ ]:
df_rolldecay.head()

In [ ]:
mask=df_rolldecay['phi_start'].abs() > np.deg2rad(5)
df_rolldecay=df_rolldecay.loc[mask].copy()

In [ ]:
duplicates =df_rolldecay.groupby(by=['loading_condition_id','ship_speed'])

In [ ]:
df_duplicates = duplicates.filter(lambda x:len(x)>1)

In [ ]:
df_duplicates.describe()

In [ ]:
interesting=['omega0','omega0_fft']
duplicates=df_duplicates.groupby(by=['loading_condition_id','ship_speed'])
mean_values = duplicates.mean().reset_index()[interesting]
std_values = duplicates.std().reset_index()[interesting]


In [ ]:
std_values.describe()

In [ ]:
index = std_values['omega0'].idxmin()

In [ ]:
for _,group in duplicates:
    pass

In [ ]:
group

In [ ]:
db = database.get_db()

In [ ]:
lowpass_filter_transformer = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
cut_transformer = CutTransformer(phi_max=np.deg2rad(5),phi_min=np.deg2rad(0.5))
offset_transformer = OffsetTransformer()

In [ ]:
def load_run(run_id:int):
    db_run = db.session.query(Run).get(int(run_id))
    df_run = database.load_run(db_run=db_run)
        
    lowpass_filter_transformer.fit(X=df_run)
    X=lowpass_filter_transformer.transform(X=df_run)
    cut_transformer.fit(X=X)
    X = cut_transformer.transform(X=X)
    offset_transformer.fit(X=X)
    X = offset_transformer.transform(X=X)
    
    X.index-=X.index[0]

    if X.iloc[0]['phi']<0:
        X['phi']*=-1
    
    return X

def plot_group(group):
    fig,ax = plt.subplots()
    
    for run_id, row in group.iterrows():
        X=load_run(run_id=run_id)
        

        
        if run_id in df_rolldecay_only_latest.index:
            plot_style='-'
        else:
            plot_style='--'
        
        X.plot(y='phi', label='run:%i' % run_id, ax=ax, style=plot_style)
    
    ax.grid()

In [ ]:
counter = 0
for _,group in duplicates:
    counter+=1
    if counter>20:
        break 
        
    plot_group(group)

In [ ]:
df_good = duplicates.filter(lambda x:x['omega0'].std()<0.0004)
df_good.describe()

In [ ]:
good_groups = df_good.groupby(by=['loading_condition_id','ship_speed'])

In [ ]:
counter = 0
for _,group in good_groups:
    counter+=1
    if counter>20:
        break 
        
    plot_group(group)

In [ ]:
def r2_group(group, plot=False):
    
    X1=load_run(run_id=group.index[0])
    X2=load_run(run_id=group.index[1])
    
    resample_time='0.05S'
    X1.index = pd.TimedeltaIndex(X1.index, unit='s')
    X1=X1.resample(resample_time).mean()
    
    X2.index = pd.TimedeltaIndex(X2.index, unit='s')
    X2=X2.resample(resample_time).mean()
    
    df_compare = pd.merge(left=X1, right=X2, how='inner', left_index=True, right_index=True,
                         suffixes=('_1','_2'))
    
    if plot:
        df_compare.plot(y=['phi_1','phi_2'])
        
    return r2_score(y_true=df_compare['phi_1'], y_pred=df_compare['phi_2'])    
    
                


In [ ]:
r2s = good_groups.apply(func=r2_group, plot=False)

In [ ]:
r2s

In [ ]:
r2s.mean()

In [ ]:
r2s.std()